In [1]:
# Install required libraries
!pip install PyPDF2

import pandas as pd
import json
from PyPDF2 import PdfReader


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [3]:
# Step 1: Upload and extract text from PDF
from google.colab import files



# Get the uploaded file path
pdf_path = "/content/FORM 1040 Instructions.pdf"

# Step 2: Extract text from the uploaded PDF
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = []
    for page in reader.pages:
        text.append(page.extract_text())
    return "\n".join(text)

# Extract text
pdf_text = extract_text_from_pdf(pdf_path)

In [4]:
# Step 2: Save extracted text to JSONL
filename = "Raw_Data.jsonl"
data = [{"question": f"What is on page {i + 1}?", "answer": page_text}
        for i, page_text in enumerate(pdf_text.split("\n\n"))]  # Splitting pages for example
with open(filename, "w") as jsonl_file:
    for item in data:
        jsonl_file.write(json.dumps(item) + "\n")

In [5]:
# Step 3: Load the JSONL as DataFrame
instruction_dataset_df = pd.read_json(filename, lines=True)
examples = instruction_dataset_df.to_dict()


In [6]:
# Step 4: Construct fine-tuning dataset
if "question" in examples and "answer" in examples:
    text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
    text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
    text = examples["input"][0] + examples["output"][0]
else:
    text = examples["text"][0]

prompt_template = """### Question:
{question}

### Answer:"""

num_examples = len(examples["question"])
finetuning_dataset = []
for i in range(num_examples):
    question = examples["question"][i]
    answer = examples["answer"][i]
    text_with_prompt_template = prompt_template.format(question=question)
    finetuning_dataset.append({"question": text_with_prompt_template, "answer": answer})
